In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import scipy.sparse

# 1. データ読み込み
ratings = pd.read_csv("ratings.csv") # ここのシステムを考え直す（評価が変わるたびにcsvをダウンロードするわけにはいかない）
movies  = pd.read_csv("tmdb_5000_movies.csv") #　ここも、新しい映画が公開されるたびにダウンロードするわけにはいかない

# 1.1 列名確認（moviesがどんなカラムを持っているかをチェック）
print("columns:", movies.columns)

# 2. ID 列が 'id' なら 'movieId' にリネーム
if "id" in movies.columns:
    movies = movies.rename(columns={"id": "movieId"})
movies["movieId"] = pd.to_numeric(movies["movieId"], errors="coerce")

# 3. 欠損値（NaN） を空文字に置き換え
# 前までは、movies["genres"] = movies["genres"].fillna("")、これはジャンルだけだったから
for col in ["genres", "keyword", "overview", "original_language"]:
    if col in movies.columns:
        movies[col] = movies[col].fillna("")


# 4. TF–IDF ベクトル化
vec = TfidfVectorizer(
    stop_words="english",
    max_features=5000,
    ngram_range=(1,3)
    )
tfidf_matrix = vec.fit_transform(movies["overview"].tolist()) # 映画ごとのあらすじを（映画数✖️5000）の行列に変換し、単語の重要度を計算
print("TF-IDF matrix shape:", tfidf_matrix.shape)

# 5. genresを文字列からリストに変換し、さらにワンホット行列にする
movies["genres_list"] = (
    movies["genres"]
    .apply(ast.literal_eval)
    .apply(lambda L: [d["name"] for d in L])
)
mlb = MultiLabelBinarizer()
genre_mat = mlb.fit_transform(movies["genres_list"])
print("Genres one-hot shape:", genre_mat.shape)
print("Genre classes:", mlb.classes_)

# 6. TF-IDF行列とジャンルワンホットを結合する
features = scipy.sparse.hstack([tfidf_matrix, genre_mat]).tocsc()
print("combined features shape:",features.shape)

# 7. 好きな映画インデックス取得 (★4以上)
liked = ratings[ratings["rating"] >= 4]["movieId"].tolist()
liked_indices = movies[movies["movieId"].isin(liked)].index.tolist()

# 8. ユーザーベクトル作成
user_vec = features[liked_indices].mean(axis=0)
user_vec = np.asarray(user_vec)
print("User vector shape:", user_vec.shape)

# 9. コサイン類似度計算＆評価済み除外
sims = cosine_similarity(user_vec, features).flatten()
for idx in liked_indices:
    sims[idx] = -1

# 10. 上位5件を抽出・表示
top5_idx = np.argsort(sims)[::-1][:5]
recommendations = movies.iloc[top5_idx][["movieId", "title", "genres_list"]]
recommendations.reset_index(drop=True, inplace=True)
display(recommendations)

columns: Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')
TF-IDF matrix shape: (4803, 5000)
Genres one-hot shape: (4803, 20)
Genre classes: ['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary' 'Drama'
 'Family' 'Fantasy' 'Foreign' 'History' 'Horror' 'Music' 'Mystery'
 'Romance' 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
combined features shape: (4803, 5020)
User vector shape: (1, 5020)


,movieId,title,genres_list
0,8883,Flame & Citron,"[Crime, Drama, History, War]"
1,238,The Godfather,"[Drama, Crime]"
2,3131,Gangs of New York,"[Drama, History, Crime]"
3,524,Casino,"[Drama, Crime]"
4,88794,J. Edgar,"[Drama, Crime, History]"
